# Example Model Servers with Seldon

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core>=0.2.6.1```)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

In [36]:
!kubectl create namespace seldon

namespace/seldon created


In [37]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "gke_seldon-demos_europe-west6-b_standard-cluster-1" modified.


In [38]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


## Install Helm

In [39]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [40]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [41]:
!helm install ../helm-charts/seldon-core-operator --name seldon-core  --set usageMetrics.enabled=true --namespace seldon-system

NAME:   seldon-core
LAST DEPLOYED: Sun Jul 14 15:56:47 2019
NAMESPACE: seldon-system
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRole
NAME                          AGE
seldon-operator-manager-role  1s

==> v1/ClusterRoleBinding
NAME                                 AGE
seldon-operator-manager-rolebinding  1s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  1     1s

==> v1/Pod(related)
NAME                                         READY  STATUS             RESTARTS  AGE
seldon-operator-controller-manager-0         0/1    ContainerCreating  0         0s
seldon-spartakus-volunteer-5b568c587b-wpbl2  0/1    ContainerCreating  0         0s

==> v1/Secret
NAME                                   TYPE    DATA  AGE
seldon-operator-webhook-server-secret  Opaque  0     1s

==> v1/Service
NAME                                        TYPE       CLUSTER-IP    EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10.78.7.11    <none>       443/TCP  0

In [10]:
!kubectl rollout status statefulset.apps/seldon-operator-controller-manager -n seldon-system

partitioned roll out complete: 1 new pods have been updated...


## Setup Ingress
There are gRPC issues with the latest Ambassador, so we rewcommend 0.40.2 until these are fixed.

In [42]:
!helm install stable/ambassador --name ambassador --set crds.keep=false

NAME:   ambassador
LAST DEPLOYED: Sun Jul 14 16:00:55 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Deployment
NAME        READY  UP-TO-DATE  AVAILABLE  AGE
ambassador  0/3    3           0          1s

==> v1/Pod(related)
NAME                         READY  STATUS             RESTARTS  AGE
ambassador-67ffd4f6c5-4nwbv  0/1    ContainerCreating  0         1s
ambassador-67ffd4f6c5-gwhhh  0/1    ContainerCreating  0         1s
ambassador-67ffd4f6c5-rcbfc  0/1    ContainerCreating  0         1s

==> v1/Service
NAME               TYPE          CLUSTER-IP    EXTERNAL-IP  PORT(S)                     AGE
ambassador         LoadBalancer  10.78.0.56    <pending>    80:31414/TCP,443:31806/TCP  1s
ambassador-admins  ClusterIP     10.78.10.131  <none>       8877/TCP                    1s

==> v1/ServiceAccount
NAME        SECRETS  AGE
ambassador  1        1s

==> v1beta1/ClusterRole
NAME        AGE
ambassador  1s

==> v1beta1/ClusterRoleBinding
NAME        AGE
ambassador  1s

==> v1bet

In [43]:
!kubectl rollout status deployment.apps/ambassador

Waiting for deployment "ambassador" rollout to finish: 0 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 1 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 2 of 3 updated replicas are available...
deployment "ambassador" successfully rolled out


### Port Forward to Ambassador

```
kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Serve SKlearn Iris Model

In [1]:
!pygmentize ../servers/sklearnserver/samples/iris.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1


In [22]:
!kubectl apply -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io/sklearn created


In [23]:
!kubectl rollout status deploy/iris-default-8bb3ef6

deployment "iris-default-8bb3ef6" successfully rolled out


In [5]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="sklearn",namespace="seldon")

In [6]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.29275055986695164
    values: 0.8118062883127178
    values: 0.19700994587239462
    values: 0.6118673849089248
  }
}

Response:
meta {
  puid: "bhj2u94r08iqvoq3p18i98328l"
  requestPath {
    key: "classifier"
    value: "seldonio/sklearnserver_rest:0.1"
  }
}
data {
  tensor {
    shape: 1
    values: 0.0
  }
}



In [24]:
!kubectl delete -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io "sklearn" deleted


## Serve XGBoost Iris Model

In [8]:
!pygmentize ../servers/xgboostserver/samples/iris.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: xgboost
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: XGBOOST_SERVER
      modelUri: gs://seldon-models/xgboost/iris
      name: classifier
    name: default
    replicas: 1


In [9]:
!kubectl apply -f ../servers/xgboostserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io/xgboost created


In [10]:
!kubectl rollout status deploy/iris-default-5299e79

Waiting for deployment "iris-default-5299e79" rollout to finish: 0 of 1 updated replicas are available...
deployment "iris-default-5299e79" successfully rolled out


In [11]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="xgboost",namespace="seldon")

In [12]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.8051608636875349
    values: 0.8477511606179198
    values: 0.2803798056079224
    values: 0.2967822556594124
  }
}

Response:
meta {
  puid: "8kn3r98jan6cunnta26rep16p4"
  requestPath {
    key: "classifier"
    value: "seldonio/xgboostserver_rest:0.1"
  }
}
data {
  tensor {
    shape: 1
    values: 0.0
  }
}



In [13]:
!kubectl delete -f ../servers/xgboostserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io "xgboost" deleted


## Serve Tensorflow MNIST Model

**Will only work on a GCP Kubernetes Cluster**

In [14]:
!pygmentize ../servers/tfserving/samples/mnist_rest.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: tfserving
spec:
  name: mnist
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/mnist-model
      name: mnist-model
      parameters:
        - name: signature_name
          type: STRING
          value: predict_images
        - name: model_name
          type: STRING
          value: mnist-model
    name: default
    replicas: 1


In [15]:
!kubectl apply -f ../servers/tfserving/samples/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io/tfserving created


In [16]:
!kubectl rollout status deploy/mnist-default-d53c803

Waiting for deployment "mnist-default-d53c803" rollout to finish: 0 of 1 updated replicas are available...
deployment "mnist-default-d53c803" successfully rolled out


In [18]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="tfserving",namespace="seldon")

In [19]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,784))
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 784
    values: 0.4616407609269467
    values: 0.724579128885353
    values: 0.10772363066533641
    values: 0.9622370745070161
    values: 0.38107285587704454
    values: 0.9129452966801331
    values: 0.30533800703940484
    values: 0.6013002423394798
    values: 0.79026885623308
    values: 0.631489736295451
    values: 0.49846618943383936
    values: 0.5303189065199159
    values: 0.58944694862469
    values: 0.39633977607259374
    values: 0.6424230610027857
    values: 0.7906748163388911
    values: 0.28558106611583367
    values: 0.021081907753274676
    values: 0.7465477978887378
    values: 0.5847521513217439
    values: 0.4805783482036078
    values: 0.6560854742284569
    values: 0.4585105200940378
    values: 0.23285913028376937
    values: 0.4423508606448364
    values: 0.325825666173277
    values: 0.6755349982171834
    values: 0.9171111975521336
    values: 0.9309797800170231
    values: 0.02579248

In [21]:
!kubectl delete -f ../servers/tfserving/samples/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io "tfserving" deleted
